# Text Representation

This notebook covers different methods for representing text as numerical vectors that machine learning models can understand:
- **Bag of Words (BOW)**: Simple word frequency representation
- **Count Vectorizer**: Scikit-learn implementation of BOW
- **TF-IDF (Term Frequency-Inverse Document Frequency)**: Weighted word importance
- **Word2Vec**: Word embeddings using neural networks
- **Modern Embeddings**: Overview of transformer-based embeddings

## Learning Objectives

- Understand different text representation techniques
- Convert text to numerical vectors using various methods
- Compare and contrast different representation approaches
- Apply embeddings for semantic similarity and clustering


## Installation

Run this cell to install required packages (uncomment if needed):


In [ ]:
# Install packages (uncomment if needed)
# !pip install scikit-learn gensim numpy pandas matplotlib seaborn


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


## Sample Documents

We'll use these sample documents to demonstrate different text representation methods:


In [ ]:
# Sample documents
documents = [
    "Machine learning is a subset of artificial intelligence",
    "Artificial intelligence and machine learning are transforming technology",
    "Deep learning is a type of machine learning",
    "Natural language processing uses machine learning",
    "Computer vision and image processing are important AI fields"
]

print("Sample Documents:")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc}")


# 1. Bag of Words (BOW)

Bag of Words is the simplest text representation method. It creates a vocabulary of all unique words and represents each document as a vector of word counts.

## 1.1 Manual BOW Implementation


In [ ]:
# Manual BOW implementation
def create_bow(documents):
    """Create a simple Bag of Words representation"""
    # Build vocabulary
    vocabulary = set()
    for doc in documents:
        words = doc.lower().split()
        vocabulary.update(words)
    
    vocabulary = sorted(list(vocabulary))
    
    # Create BOW vectors
    bow_vectors = []
    for doc in documents:
        words = doc.lower().split()
        vector = [words.count(word) for word in vocabulary]
        bow_vectors.append(vector)
    
    return vocabulary, np.array(bow_vectors)

vocab, bow_matrix = create_bow(documents)

print("Vocabulary:", vocab)
print("\nBOW Matrix:")
print(pd.DataFrame(bow_matrix, columns=vocab, index=[f"Doc {i+1}" for i in range(len(documents))]))


## 1.2 Count Vectorizer (Scikit-learn)

Scikit-learn's `CountVectorizer` provides a more robust BOW implementation with preprocessing options:


In [ ]:
# Initialize CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform documents
count_matrix = count_vectorizer.fit_transform(documents)

# Get vocabulary
vocabulary = count_vectorizer.get_feature_names_out()

# Convert to dense array for visualization
count_array = count_matrix.toarray()

print("Count Vectorizer Results:")
print(f"Vocabulary size: {len(vocabulary)}")
print("\nCount Matrix:")
df = pd.DataFrame(count_array, columns=vocabulary, index=[f"Doc {i+1}" for i in range(len(documents))])
print(df)


In [ ]:
# CountVectorizer with options
count_vect = CountVectorizer(
    lowercase=True,      # Convert to lowercase
    stop_words='english', # Remove stop words
    max_features=50,      # Limit vocabulary size
    ngram_range=(1, 2)   # Use unigrams and bigrams
)

count_matrix_advanced = count_vect.fit_transform(documents)
vocab_advanced = count_vect.get_feature_names_out()

print("Advanced Count Vectorizer (with stop words removal and bigrams):")
print(f"Vocabulary size: {len(vocab_advanced)}")
print(f"Sample vocabulary: {list(vocab_advanced[:10])}")


# 2. TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF weights words by their importance: frequent in a document but rare across all documents get higher weights.

**TF (Term Frequency)**: How often a term appears in a document
**IDF (Inverse Document Frequency)**: How rare a term is across all documents

$$TF\text{-}IDF(t, d) = TF(t, d) \times IDF(t)$$

$$IDF(t) = \log\frac{N}{df(t)}$$

where N is the total number of documents and df(t) is the number of documents containing term t.


In [ ]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform documents
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Get vocabulary
tfidf_vocab = tfidf_vectorizer.get_feature_names_out()

# Convert to dense array for visualization
tfidf_array = tfidf_matrix.toarray()

print("TF-IDF Results:")
print(f"Vocabulary size: {len(tfidf_vocab)}")
print("\nTF-IDF Matrix:")
df_tfidf = pd.DataFrame(tfidf_array, columns=tfidf_vocab, index=[f"Doc {i+1}" for i in range(len(documents))])
print(df_tfidf.round(3))


In [ ]:
# Show top terms for each document
print("Top TF-IDF Terms per Document:")
for i, doc in enumerate(documents):
    # Get TF-IDF scores for this document
    scores = tfidf_array[i]
    # Get indices of top 5 terms
    top_indices = scores.argsort()[-5:][::-1]
    top_terms = [(tfidf_vocab[idx], scores[idx]) for idx in top_indices]
    
    print(f"\nDoc {i+1}: {doc[:50]}...")
    for term, score in top_terms:
        print(f"  {term}: {score:.3f}")


In [ ]:
# Calculate document similarity using TF-IDF
similarity_matrix = cosine_similarity(tfidf_matrix)

print("Document Similarity Matrix (Cosine Similarity):")
df_sim = pd.DataFrame(similarity_matrix, 
                     index=[f"Doc {i+1}" for i in range(len(documents))],
                     columns=[f"Doc {i+1}" for i in range(len(documents))])
print(df_sim.round(3))

# Visualize similarity matrix
plt.figure(figsize=(8, 6))
sns.heatmap(similarity_matrix, annot=True, fmt='.2f', cmap='YlOrRd', 
            xticklabels=[f"Doc {i+1}" for i in range(len(documents))],
            yticklabels=[f"Doc {i+1}" for i in range(len(documents))])
plt.title('Document Similarity (TF-IDF Cosine Similarity)')
plt.tight_layout()
plt.show()


# 3. Word2Vec

Word2Vec creates dense vector representations (embeddings) where similar words have similar vectors. It captures semantic relationships.

Two architectures:
- **CBOW (Continuous Bag of Words)**: Predicts word from context
- **Skip-gram**: Predicts context from word


In [ ]:
# Prepare text data for Word2Vec (tokenized sentences)
tokenized_docs = [doc.lower().split() for doc in documents]

print("Tokenized Documents:")
for i, tokens in enumerate(tokenized_docs, 1):
    print(f"Doc {i}: {tokens}")


In [ ]:
# Train Word2Vec model
# Parameters:
# - size: dimension of word vectors (embeddings)
# - window: maximum distance between current and predicted word
# - min_count: ignore words with frequency < min_count
# - sg: 0 for CBOW, 1 for Skip-gram

word2vec_model = Word2Vec(
    sentences=tokenized_docs,
    vector_size=100,      # Dimension of word vectors
    window=5,             # Context window size
    min_count=1,          # Minimum word frequency
    workers=4,            # Number of threads
    sg=0                  # 0 = CBOW, 1 = Skip-gram
)

print("Word2Vec model trained!")
print(f"Vocabulary size: {len(word2vec_model.wv.key_to_index)}")
print(f"Vector dimension: {word2vec_model.wv.vector_size}")


In [ ]:
# Get word vectors
words_to_check = ['machine', 'learning', 'artificial', 'intelligence']

print("Word Vectors (first 10 dimensions):")
for word in words_to_check:
    if word in word2vec_model.wv:
        vector = word2vec_model.wv[word]
        print(f"\n{word}: {vector[:10]}... (shape: {vector.shape})")
    else:
        print(f"\n{word}: Not in vocabulary")


In [ ]:
# Find most similar words
test_words = ['machine', 'learning', 'artificial']

print("Most Similar Words:")
for word in test_words:
    if word in word2vec_model.wv:
        similar = word2vec_model.wv.most_similar(word, topn=5)
        print(f"\nWords similar to '{word}':")
        for similar_word, score in similar:
            print(f"  {similar_word}: {score:.3f}")


In [ ]:
# Word analogies (e.g., king - man + woman ≈ queen)
# This demonstrates semantic relationships captured by Word2Vec

print("Word Analogies:")
print("Note: With limited training data, results may not be perfect")

# Example: learning - machine + artificial ≈ ?
if all(word in word2vec_model.wv for word in ['learning', 'machine', 'artificial']):
    result = word2vec_model.wv.most_similar(
        positive=['learning', 'artificial'],
        negative=['machine'],
        topn=3
    )
    print(f"\n'learning' - 'machine' + 'artificial' ≈ ?")
    for word, score in result:
        print(f"  {word}: {score:.3f}")


In [ ]:
# Get document embeddings by averaging word vectors
def get_document_embedding(doc_tokens, model):
    """Get document embedding by averaging word vectors"""
    words = [word for word in doc_tokens if word in model.wv]
    if len(words) == 0:
        return None
    vectors = [model.wv[word] for word in words]
    return np.mean(vectors, axis=0)

# Get embeddings for all documents
doc_embeddings = []
for i, tokens in enumerate(tokenized_docs):
    embedding = get_document_embedding(tokens, word2vec_model)
    if embedding is not None:
        doc_embeddings.append(embedding)
        print(f"Doc {i+1} embedding shape: {embedding.shape}")
    else:
        print(f"Doc {i+1}: No valid words found")

doc_embeddings = np.array(doc_embeddings)
print(f"\nDocument embeddings matrix shape: {doc_embeddings.shape}")


# 4. Modern Embeddings Overview

Modern NLP uses transformer-based models that generate contextual embeddings. Here's an overview:


## 4.1 Types of Modern Embeddings

### Contextual vs Non-Contextual

- **Non-Contextual (Word2Vec, GloVe)**: Same word always has the same embedding
  - Example: "bank" (river bank) and "bank" (financial bank) have the same vector
  
- **Contextual (BERT, GPT, etc.)**: Same word has different embeddings based on context
  - Example: "bank" in "river bank" vs "bank account" have different embeddings

### Popular Embedding Models

1. **Word2Vec** (2013): Word-level embeddings, non-contextual
2. **GloVe** (2014): Global word co-occurrence statistics
3. **ELMo** (2018): Contextual embeddings using bidirectional LSTMs
4. **BERT** (2018): Bidirectional transformer, contextual embeddings
5. **GPT** (2018+): Generative transformer models
6. **Sentence Transformers** (2019+): Optimized for sentence-level embeddings


In [ ]:
# Example: Using sentence-transformers (install with: pip install sentence-transformers)
# This is commented out but shows how to use modern embeddings

"""
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(documents)

print(f"Embedding shape: {embeddings.shape}")
print(f"Each document is represented as a {embeddings.shape[1]}-dimensional vector")

# Calculate similarity
similarity = cosine_similarity(embeddings)
print("\nDocument Similarity (using Sentence Transformers):")
print(pd.DataFrame(similarity, 
                   index=[f"Doc {i+1}" for i in range(len(documents))],
                   columns=[f"Doc {i+1}" for i in range(len(documents))]).round(3))
"""
print("Note: Install sentence-transformers to use modern embeddings")
print("pip install sentence-transformers")


# 5. Comparison and When to Use What

## Comparison Table

| Method | Dimensions | Contextual | Speed | Use Case |
|--------|-----------|------------|-------|----------|
| **BOW/Count** | Vocabulary size | ❌ No | ⚡ Very Fast | Simple classification, baseline |
| **TF-IDF** | Vocabulary size | ❌ No | ⚡ Fast | Information retrieval, search |
| **Word2Vec** | 100-300 | ❌ No | ⚡ Fast | Word similarity, analogies |
| **BERT/Transformers** | 768-1024+ | ✅ Yes | 🐌 Slow | Semantic understanding, modern NLP |

## Recommendations

- **BOW/TF-IDF**: 
  - Good for: Simple classification, baseline models, small datasets
  - Bad for: Semantic understanding, capturing word relationships
  
- **Word2Vec**: 
  - Good for: Word similarity, analogies, when you need word-level embeddings
  - Bad for: Context-dependent meanings, capturing sentence semantics
  
- **Modern Embeddings (BERT, etc.)**: 
  - Good for: Semantic search, understanding context, state-of-the-art results
  - Bad for: Real-time applications (unless optimized), very large datasets without GPU


In [ ]:
# Visualize the difference in dimensionality
methods = ['BOW', 'TF-IDF', 'Word2Vec', 'BERT']
vocab_size = len(vocabulary)
dimensions = [vocab_size, vocab_size, 100, 768]

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(methods, dimensions, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax.set_ylabel('Vector Dimension', fontsize=12)
ax.set_title('Comparison of Text Representation Dimensions', fontsize=14, fontweight='bold')
ax.set_yscale('log')  # Log scale to better show differences

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print(f"Note: BOW/TF-IDF dimensions = vocabulary size ({vocab_size} words)")
print("Word2Vec typically uses 100-300 dimensions")
print("BERT uses 768-1024 dimensions")


## Summary

This notebook covered:
- ✅ **Bag of Words (BOW)**: Simple word frequency representation
- ✅ **Count Vectorizer**: Scikit-learn implementation with preprocessing options
- ✅ **TF-IDF**: Weighted word importance for better document representation
- ✅ **Word2Vec**: Neural network-based word embeddings
- ✅ **Modern Embeddings**: Overview of transformer-based contextual embeddings

### Key Takeaways

1. **BOW/TF-IDF** are sparse, high-dimensional representations suitable for traditional ML
2. **Word2Vec** creates dense, low-dimensional embeddings capturing word relationships
3. **Modern embeddings** (BERT, etc.) provide contextual understanding but are computationally expensive
4. Choose the right representation based on your task, data size, and computational resources
